# IOER-RDC and NFDI data example

This section introduces a relatively linear walkthrough example. We will demonstrate a spatial data processing example from start to finish. We won't take alternative routes. Where further information is recommended, cross-references to the reference documentation (Part III) are provided. 

This section is based on two data sources:
- [NFDI Gbif biodiversity data](https://www.nfdi4biodiversity.org/en/services/): Species observations as coordinates (vector data)
- [IOER land use data](https://www.ioer-monitor.de/): Aggregated and processed BKI data that is provided in raster format (WCS)

We will retrieve data from both sources and then use intersection and aggregation methods to create an integrated visualisation.

## NFDI-Data – How and where to find the data?


The NFDI4Biodiversity Species Observations data are available at https://land.gbif.de/.

There are two ways to download data:
- By searching and downloading individual data files
- By using the [Species Application Programming Interface (API)](https://techdocs.gbif.org/en/openapi/v1/species) to programmatically access data

In both cases and for larger downloads, registration is necessary. However, we can have a brief look at a small example query by using the Species API without a key. Let's find out about the English Sparrow, before we jump in to a more detailed dataset.

Let's assume we want to find out the Scientific name, but we only have the common name for a species, e.g. the `English Sparrow`. We may hit Google. But then, what if we have a list of 500 species, or 5000 (etc.)?

By trying, we can see that the gbif.org search provides the correct answer: [https://www.gbif.org/species/search?q=english%20sparrow](https://www.gbif.org/species/search?q=english%20sparrow).

gbif.org internally also use their own API, so we may be able, too.

### NFDI-API

```{figure} https://images.naturalis.nl/original/csr%2FD80_8153.jpg
:name: sparrow-graphic

The English Sparrow (GBIF.org).
```

- Retrieving data from APIs requires a specific syntax that is different for each service.
- commonly, there is an **endpoint** (a url) that returns data in a structured format (e.g. **json**, see [Part III: Common spatial file formats](302_file_formats))
- most APIs require you to authenticate, but not all (e.g. Instagram, commons.wikimedia.org)

```{dropdown} How do I know how to work with APIs?
Good APIs have documentation. The gbif.org API documentation even allows to try the API directly. Go to <a href="https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames">https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames</a> and try the general species search for "English Sparrow". Enter this text in the field for `q (query)` (Simple full text search parameter.).
```

We can do the same in Python. Load gbif search answer for a specific common name search. Get the scientific names matching "English Sparrow".

In [1]:
search_name = "English Sparrow"
query_url = f'https://api.gbif.org/v1/species/search?q={search_name}'

(content:references:admonition)=
```{admonition} Use your own common name!
:class: dropdown, attention
Optionally replace "English Sparrow" with another location above
```

```{admonition} **Syntax: f'{}' ?**
:class: tip, dropdown
This is called an f-string, [a convenient python convention](https://realpython.com/python-f-strings/) to concat strings and variables.
```

Get the json-answer from the API (without login!).

In [2]:
import requests

json_text = None
response = requests.get(url=query_url)

The data can be found in `response.text` (in case the API answered).

In [3]:
import json
json_data = json.loads(response.text)

We can see that the correct answer, `Passer domesticus`, is hidden in the nested json structure:

In [4]:
print(json.dumps(json_data, indent=2)[0:550])

{
  "offset": 0,
  "limit": 20,
  "endOfRecords": true,
  "count": 5,
  "results": [
    {
      "key": 100220560,
      "datasetKey": "b351a324-77c4-41c9-a909-f30f77268bc4",
      "nubKey": 5231190,
      "scientificName": "Passer domesticus",
      "canonicalName": "Passer domesticus",
      "nameType": "SCIENTIFIC",
      "taxonomicStatus": "ACCEPTED",
      "origin": "SOURCE",
      "numDescendants": 0,
      "numOccurrences": 0,
      "taxonID": "420",
      "habitats": [],
      "nomenclaturalStatus": [],
      "threatStatuses": [],
     


We can access the most relevant first result and print this name directly. Below, we walk the json path from the `"results"`, which is a list, access the first entry by using `[0]` and the access its key of the name `scientificName`.

In [5]:
json_data["results"][0]["scientificName"]

'Passer domesticus'

We can also see the taxon id, a unique reference within the GBIF database that we will need in the next chapter to query occurrences:
(content:references:passer-id)=

In [6]:
json_data["results"][0]["taxonID"]

'420'

`````{admonition} Continue in the next section..
:class: seealso
Passer domesticus will be our example species for which we will retrieve observations [in the next section](202_data_retrieval).
`````

## IÖR-Data – How and where to find the data?


The IÖR-Monitor Data is available at [https://monitor.ioer.de/](https://monitor.ioer.de/). All IÖR-Monitor data is available as Web Feature Service (WFS) and Web Coverage Service (WCS).
You can see a full list of services [here](https://monitor.ioer.de/monitor_api/services).

```{figure} https://www.ioer-monitor.de/fileadmin/user_upload/monitor/img/Ergebnisse/siedlungsdichte.png
:name: monitor-graphic

Wie dicht leben wir? [IÖR Monitor data](https://www.ioer-monitor.de/ergebnisse/analyseergebnisse/wie-dicht-leben-wir/).
```

```{admonition} Registration necessary
:class: warning
To retrieve data from the IÖR-Monitor API, [registration is necessary](https://www.ioer-monitor.de/login/).

In order to follow the steps below, you need to:
1. Register [monitor.ioer.de/monitor_api/signup](https://monitor.ioer.de/monitor_api/signup),
2. Create an API key,
3. Store this API key in a file called `.env` as `API_KEY=xyz`
```

```{admonition} Continue in the next section..
:class: seealso
[See the next section](202_data_retrieval) where we explain how to access the IÖR-WCS raster data for green cover in Saxony.
```

## Example research question

Now we have a research question. In the following example workflow, we investigate whether there is a positive correlation between sparrow density and settlement areas. Since sparrows are known to prefer settled areas, this should be visible with the IÖR monitor and the NFDI4Biodiversity data.

Furthermore, we want to see if there is a population trend for the sparrow in Saxony. {cite:ts}`salek_house_2015` observed an overall decreasing population trend for the sparrow.

> _Populations of House and Tree Sparrows have rapidly declined in various breeding habitats throughout their European distribution range; however, the strongest decline was recorded within urban environments._
{cite:p}`salek_house_2015`

Is this also true for Saxony? [Let's see..](202_data_retrieval)

## References

```{bibliography}
:style: unsrt
:filter: docname in docnames
```